<a href="https://colab.research.google.com/github/beeguy74/Ai-ta-place/blob/main/gradio_stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.6 MB/s e

In [ ]:
from transformers import pipeline
from transformers import BarkModel, BarkProcessor

In [ ]:
import gradio as gr
from pydub import AudioSegment
from time import sleep


In [ ]:
# synthesiser = pipeline("text-to-speech", "suno/bark-small", device=0) #pipeline("text-to-speech")

# model = BarkModel.from_pretrained("suno/bark-small")
model = BarkModel.from_pretrained("suno/bark-small").to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.cl

In [ ]:
processor = BarkProcessor.from_pretrained("suno/bark-small", device=0)
# processor = BarkProcessor.from_pretrained("suno/bark-small")

In [ ]:
import sys
from IPython.display import Audio
from numpy import squeeze

def create_audio(text, lang):
  sampling_rate = model.generation_config.sample_rate
  voice_preset = "v2/fr_speaker_1" if lang == "french" else "v2/en_speaker_3"
  inputs = processor(text, voice_preset=voice_preset)
  output = model.generate(**inputs.to("cuda"))
  # output = model.generate(**inputs)
  return Audio(output.cpu().numpy(), rate=sampling_rate).data

In [ ]:


with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot(
                value=[[None, "Hi! i am friendly cahtbot!"]],
                height="50vh"
            )
            text = gr.Textbox()

        def user(user_message, history):
          print("USER", history, file=sys.stderr)
          return "", history + [[user_message, None]]

        def stream_bytes(history):
            print("STREAM", history, file=sys.stderr)
            chunk_size = 1000
            sleep(0.1)
            for line in history:
              print("PROCESSING", line[0], file=sys.stderr)
              res = create_audio(line[0], "english")
              i = 0
              while i < len(res):
                  chunk = []
                  if i + chunk_size <= len(res):
                    chunk = res[i: i + chunk_size]
                  else:
                    chunk = res[i: ]
                  i += chunk_size + 1
                  if chunk:
                      yield chunk
                  else:
                      break

        with gr.Column():
            audio = gr.Audio(
                    sources=["microphone"],
                    streaming=True,
                    label="Talk to me"
            )
            stream_as_bytes_output = gr.Audio(
                autoplay=True,
                streaming=True,
                label="Listen to me"
            )
            stream_as_bytes_btn = gr.Button("Stream as Bytes")

        text.submit(user, [text, chatbot], [text, chatbot]).then(
            stream_bytes, chatbot, stream_as_bytes_output
        )
        # text.submit(user, [text, chatbot], [text, chatbot])
        # stream_as_bytes_btn.click(stream_bytes, chatbot, stream_as_bytes_output)



In [ ]:
  demo.queue().launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1fb465d82983c3a0f0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


USER [[None, 'Hi! i am friendly cahtbot!']]
STREAM [[None, 'Hi! i am friendly cahtbot!'], ['hi!', None]]
PROCESSING None
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1788, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1352, in call_function
    prediction = await utils.async_iteration(iterator)
  File "/usr/local/lib/python3.10/dist-packages/gradio/utils.py", line 583, in async_iteration
    return await iterator.__anext__()
  File "/usr/local/lib/python3.10/dist-packages/gradio/utils.py", line 576, in __anext__
    return await anyio.to_thread.run_sync

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1fb465d82983c3a0f0.gradio.live


In [ ]:

demo.stop()